In [1]:
__author__ = "Chuhang Liu"

import numpy as np
import matplotlib.pyplot as plt
import os
import imageio
import scipy.signal
import scipy
from matplotlib import figure
import h5py
import hdf5plugin
import cv2


In [96]:
# Define source dict
dirstr = 'D:\\2100f_data_DECTRIS\\YIG_MIT_20240320\\100um_s1\\Mag\\x2000\\diff\\'
# Define working dict
dest_dir = 'D:\\2100f_data_DECTRIS\\YIG_MIT_20240320\\100um_s1\\Mag\\x2000\\diff\\im\\'

In [97]:
import glob

folder_path = dirstr
file_extension = '*.h5'

file_list = glob.glob(folder_path + file_extension)

# Find the file with keyword
keyword = 'master'
# Remove the file extension
file_list = [file.rstrip('.h5') for file in file_list if keyword in file]
print(file_list)




['D:\\2100f_data_DECTRIS\\YIG_MIT_20240320\\100um_s1\\Mag\\x2000\\diff\\30cm_001_master']


In [ ]:
# for videos
n_frames = 100


In [98]:
r_crosshair = 0.8886
for file in file_list:
    with h5py.File(file + '.h5', 'r') as f:
        #print(f.keys())
        
        # Load data
        dataset = f['entry/data/data_000001']
        # Read the dataset into a NumPy array
        data = np.array(dataset)
        # For images with dead pixels
        #data[data == 4294967295] = 0
        data = data.reshape(512, 512)
        # fix the crosshair mark
        #print(data[254:258, 254:258])
        data[254:258, :] = data[254:258, :]/r_crosshair
        data[:, 254:258] = data[:, 254:258]/r_crosshair
        # detect dead pixels position
        '''
        for i in range(512):
            for j in range(512):
                
                if data[i, j] == 0:
                    print(i, j)
                    data[i, j] = (data[i-1, j] + data[i+1, j] + data[i, j-1] + data[i, j+1])/4

        '''
        data[164, 87] = (data[163, 87] + data[165, 87] + data[164, 86] + data[164, 88])/4
        data[192,85] = (data[191, 85] + data[193, 85] + data[192, 84] + data[192, 86])/4
        # max_threshold = 0.6 * max_val
        # min_threshold = 0.2 * max_val
        max_val = np.max(data)
        #print("max_val: ", max_val)
        min_val = np.min(data) 
        #print("min_val: ", min_val)
        # for image
        max_threshold = np.percentile(data, 99)
        # for diffraction
        #max_threshold = np.percentile(data, 99.9)
        #print(max_threshold)
        data[data > max_threshold] = max_threshold
        min_threshold = np.percentile(data, 1)
        #print(min_threshold)
        data[data < min_threshold] = min_threshold
        # fix dead pixels
        
        # apply thresholds

        
        # generate tiffs
        name = file.split('\\')[-1]
        # for diffraction
        #Cmap = plt.cm.turbo
        # for image
        Cmap = plt.cm.gray
        try:
            plt.imsave(dest_dir + file + '.tif', data, format='tiff', cmap=Cmap)
        except:
            os.makedirs(dest_dir, exist_ok=True)
            plt.imsave(dest_dir + name + '.tif', data, format='tiff', cmap=Cmap)